In [13]:
import requests
import pandas as pd
from google.cloud import storage

# Pre-reqs:
# 1. `pip install pandas pyarrow google-cloud-storage`
# 2. Set GOOGLE_APPLICATION_CREDENTIALS to your project/service-account key
# 3. Set BUCKET as your bucket

In [15]:
init_url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/"

service = "green"
year = "2019"
month = "01"

BUCKET = "splendid-strand-448621-u2-warehouse-bucket"

In [16]:
# csv file_name
file_name = f"{service}_tripdata_{year}-{month}.csv.gz"

# download it using requests via a pandas df
request_url = f"{init_url}{service}/{file_name}"
r = requests.get(request_url)
open(file_name, "wb").write(r.content)
print(f"Local: {file_name}")

Local: green_tripdata_2019-01.csv.gz


In [17]:
# read it back into a parquet file
df = pd.read_csv(file_name, compression="gzip")
file_name = file_name.replace(".csv.gz", ".parquet")
df.to_parquet(file_name, engine="pyarrow")
print(f"Parquet: {file_name}")

Parquet: green_tripdata_2019-01.parquet


In [18]:
# upload it to gcs
client = storage.Client()
bucket = client.bucket(BUCKET)
blob = bucket.blob(f"{service}/{file_name}")
blob.upload_from_filename(file_name)